Imports

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from os import chdir, remove, close, listdir, path
from pydub import AudioSegment 


c:\Users\rowan\OneDrive\Desktop\code\iResearch-2024\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rowan\OneDrive\Desktop\code\iResearch-2024\venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
# https://archive.liveatc.net/kjfk/KJFK-Twr-Jun-24-2024-0000Z.mp3

$baseUrl = "https://archive.liveatc.net/kjfk/KJFK-Twr-Jun-24-2024-"

# Initialize time as a string
$time = "0000"

$client = New-Object System.Net.WebClient

# Loop until time is less than 24 hours
while ($time -ne "2330") {
	$url = $baseUrl + $time + "Z.mp3"

	Write-Output "Downloading $url"

	try {
		# Download the file
		$client.DownloadFile($url, "/home/rodoggx/iResearch-2024/ATC-recordings/KJFK-Twr-Jun-24-2024-" + $time + "Z.mp3")
	} catch {
		Write-Output "File not found"
	}
    # Convert string to DateTime object for easy manipulation
    $timeObject = [DateTime]::ParseExact($time, "HHmm", $null)

    # Add 30 minutes
    $timeObject = $timeObject.AddMinutes(30)

    # Convert DateTime object back to string
    $time = $timeObject.ToString("HHmm")
}

In [6]:
chdir("/home/rodoggx/iResearch-2024/ATC-recordings")
for foo in listdir():
    input_file = foo
    output_file = foo.rsplit('.', 1)[0] + '.wav'

    # convert mp3 file to wav file 
    sound = AudioSegment.from_mp3(input_file) 
    sound.export(output_file, format="wav")
    
    remove(foo)

In [16]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

for foo in listdir():
    result = pipe(foo, return_timestamps=True)
    print(result)
    print("\n----------------------------------------------------------\n")

    with open("output.txt", "a") as f:
        f.write(str(result))
        f.write("\n----------------------------------------------------------\n")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/rodoggx/iResearch-2024/venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
